In [74]:
import pandas as pd

In [75]:
matches = pd.read_csv('matches.csv', index_col=0)

In [76]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2024-08-16,20:00,Premier League,Matchweek 1,Fri,Home,W,1,0,Fulham,...,Match Report,NaN,14.0,5.0,17.2,0.0,0,0,2024,Manchester United
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,Match Report,NaN,17.0,8.0,13.9,0.0,0,0,2024,Manchester City
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,Match Report,NaN,14.0,4.0,17.9,0.0,0,0,2024,Manchester City
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,Match Report,NaN,29.0,9.0,17.3,2.0,0,1,2024,Manchester City
5,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,W,5,1,Fulham,...,Match Report,NaN,6.0,4.0,14.8,0.0,1,1,2024,Manchester City


In [77]:
matches["date"] = pd.to_datetime(matches["date"])
matches["venue_code"] = matches["venue"].astype("category").cat.codes
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")
matches["day_code"] = matches["date"].dt.dayofweek

In [83]:
#set up a target col to know whether the team won or not, w 1 marking a win and 0 loss

matches["target"] = (matches["result"] == "W").astype("int")

In [84]:
matches

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,venue_code,opp_code,hour,day_code,target
1,2024-08-16,20:00,Premier League,Matchweek 1,Fri,Home,W,1,0,Fulham,...,0.0,0,0,2024,Manchester United,1,9,20,4,1
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,0.0,0,0,2024,Manchester City,0,5,20,4,1
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,0.0,0,0,2024,Manchester City,1,16,20,5,1
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,2.0,0,1,2024,Manchester City,0,18,14,6,1
5,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,W,5,1,Fulham,...,0.0,1,1,2024,Manchester City,1,9,15,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,2023-04-30,14:00,Premier League,Matchweek 34,Sun,Away,L,1,3,Newcastle Utd,...,0.0,0,0,2023,Southampton,0,16,14,6,0
43,2023-05-08,20:00,Premier League,Matchweek 35,Mon,Away,L,3,4,Nott'ham Forest,...,0.0,1,1,2023,Southampton,0,17,20,0,0
44,2023-05-13,15:00,Premier League,Matchweek 36,Sat,Home,L,0,2,Fulham,...,0.0,0,0,2023,Southampton,1,9,15,5,0
45,2023-05-21,14:00,Premier League,Matchweek 37,Sun,Away,L,1,3,Brighton,...,1.0,0,0,2023,Southampton,0,4,14,6,0


In [85]:
from sklearn.ensemble import RandomForestClassifier

In [86]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [87]:
train = matches[matches["date"] < '2024-01-01']
test = matches[matches["date"] > '2024-01-01']

In [129]:
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [90]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [91]:
preds = rf.predict(test[predictors])

In [92]:
from sklearn.metrics import accuracy_score #what percentage of time was prediction accurate

In [93]:
acc = accuracy_score(test["target"], preds)
acc

In [95]:
combined = pd.DataFrame(dict(actual=test["target"], prediction=preds))

In [96]:
pd.crosstab(index=combined["actual"], columns = combined["prediction"]) #two way table to show when we predicted a 0/1, what actually happened

prediction,0,1
actual,,
0,168,61
1,77,61


In [97]:
from sklearn.metrics import precision_score

In [98]:
precision_score(test["target"], preds)

0.5

In [99]:
grouped_matches = matches.groupby("team")

In [100]:
group = grouped_matches.get_group("Arsenal")

In [101]:
group

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,venue_code,opp_code,hour,day_code,target
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2,1,Nott'ham Forest,...,0.0,0,0,2024,Arsenal,1,17,12,5,1
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1,0,Crystal Palace,...,0.0,1,1,2024,Arsenal,0,7,20,0,1
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2,2,Fulham,...,0.0,1,1,2024,Arsenal,1,9,15,5,0
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3,1,Manchester Utd,...,0.0,0,0,2024,Arsenal,1,15,16,6,1
5,2023-09-17,16:30,Premier League,Matchweek 5,Sun,Away,W,1,0,Everton,...,0.0,0,0,2024,Arsenal,0,8,16,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,2023-05-02,20:00,Premier League,Matchweek 34,Tue,Home,W,3,1,Chelsea,...,0.0,0,0,2023,Arsenal,1,6,20,1,1
45,2023-05-07,16:30,Premier League,Matchweek 35,Sun,Away,W,2,0,Newcastle Utd,...,0.0,0,0,2023,Arsenal,0,16,16,6,1
46,2023-05-14,16:30,Premier League,Matchweek 36,Sun,Home,L,0,3,Brighton,...,0.0,0,0,2023,Arsenal,1,4,16,6,0
47,2023-05-20,17:30,Premier League,Matchweek 37,Sat,Away,L,0,1,Nott'ham Forest,...,0.0,0,0,2023,Arsenal,0,17,17,5,0


In [102]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date") #sort in ascending order by date
    rolling_stats = group[cols].rolling(3, closed='left').mean() #closed = left keeps current week out of that week's pred
    group[new_cols] = rolling_stats
    #need to drop missing values, removes rows w missing values
    group = group.dropna(subset=new_cols)
    return group
    

In [118]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

In [119]:
new_cols

['gf_rolling',
 'ga_rolling',
 'sh_rolling',
 'sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling']

In [120]:
rolling_averages(group, cols, new_cols)

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2,1,Fulham,...,5,1,3.000000,0.666667,14.333333,5.000000,14.133333,0.333333,0.000000,0.000000
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2,1,Aston Villa,...,2,1,3.000000,1.000000,18.333333,7.000000,14.433333,0.333333,0.000000,0.000000
5,2022-09-04,16:30,Premier League,Matchweek 6,Sun,Away,L,1,3,Manchester Utd,...,6,0,2.333333,0.666667,19.333333,7.333333,15.533333,0.666667,0.000000,0.000000
7,2022-09-18,12:00,Premier League,Matchweek 8,Sun,Away,W,3,0,Brentford,...,6,1,1.666667,1.666667,20.000000,6.333333,16.800000,1.000000,0.000000,0.000000
8,2022-10-01,12:30,Premier League,Matchweek 9,Sat,Home,W,3,1,Tottenham,...,5,1,2.000000,1.333333,17.000000,6.000000,17.700000,0.666667,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,2024-04-23,20:00,Premier League,Matchweek 29,Tue,Home,W,5,0,Chelsea,...,1,1,1.666667,0.666667,20.333333,6.333333,15.800000,0.333333,0.333333,0.333333
48,2024-04-28,14:00,Premier League,Matchweek 35,Sun,Away,W,3,2,Tottenham,...,6,1,2.333333,0.666667,23.000000,7.666667,17.100000,0.333333,0.000000,0.000000
49,2024-05-04,12:30,Premier League,Matchweek 36,Sat,Home,W,3,0,Bournemouth,...,5,1,3.333333,0.666667,20.000000,7.333333,16.133333,0.333333,0.000000,0.000000
50,2024-05-12,16:30,Premier League,Matchweek 37,Sun,Away,W,1,0,Manchester Utd,...,6,1,3.666667,0.666667,20.000000,7.000000,14.866667,0.333333,0.333333,0.333333


In [121]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

In [122]:
matches_rolling = matches_rolling.droplevel("team")

In [123]:
# to drop nonunique values, keeps unique values for each index rather than repeating

matches_rolling.index = range(matches_rolling.shape[0])

In [124]:
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
0,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2,1,Fulham,...,5,1,3.000000,0.666667,14.333333,5.000000,14.133333,0.333333,0.000000,0.000000
1,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2,1,Aston Villa,...,2,1,3.000000,1.000000,18.333333,7.000000,14.433333,0.333333,0.000000,0.000000
2,2022-09-04,16:30,Premier League,Matchweek 6,Sun,Away,L,1,3,Manchester Utd,...,6,0,2.333333,0.666667,19.333333,7.333333,15.533333,0.666667,0.000000,0.000000
3,2022-09-18,12:00,Premier League,Matchweek 8,Sun,Away,W,3,0,Brentford,...,6,1,1.666667,1.666667,20.000000,6.333333,16.800000,1.000000,0.000000,0.000000
4,2022-10-01,12:30,Premier League,Matchweek 9,Sat,Home,W,3,1,Tottenham,...,5,1,2.000000,1.333333,17.000000,6.000000,17.700000,0.666667,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447,2024-04-24,19:45,Premier League,Matchweek 29,Wed,Home,L,0,1,Bournemouth,...,2,0,1.000000,2.000000,9.666667,4.000000,20.533333,0.333333,0.333333,0.333333
1448,2024-04-27,15:00,Premier League,Matchweek 35,Sat,Home,W,2,1,Luton Town,...,5,1,0.666667,1.666667,10.333333,3.333333,18.766667,0.000000,0.000000,0.000000
1449,2024-05-04,17:30,Premier League,Matchweek 36,Sat,Away,L,1,5,Manchester City,...,5,0,0.666667,1.333333,11.000000,4.000000,19.666667,0.000000,0.000000,0.000000
1450,2024-05-11,15:00,Premier League,Matchweek 37,Sat,Home,L,1,3,Crystal Palace,...,5,0,1.000000,2.333333,10.000000,3.333333,15.966667,0.000000,0.000000,0.000000


In [133]:
print(matches_rolling.columns)

Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain', 'formation',
       'opp formation', 'referee', 'match report', 'notes', 'sh', 'sot',
       'dist', 'fk', 'pk', 'pkatt', 'season', 'team', 'venue_code', 'opp_code',
       'hour', 'day_code', 'target', 'gf_rolling', 'ga_rolling', 'sh_rolling',
       'sot_rolling', 'dist_rolling', 'fk_rolling', 'pk_rolling',
       'pkatt_rolling'],
      dtype='object')


In [134]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2024-01-01']
    test = data[data["date"] > '2024-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    precision = precision_score(test["target"], preds)
    return combined, precision


In [135]:
col_list = ["venue_code", "opp_code", "hour", "day_code", 'gf_rolling',
 'ga_rolling',
 'sh_rolling',
 'sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling']

In [142]:
combined, precision = make_predictions(matches_rolling, col_list)

In [140]:
precision

0.5483870967741935

In [141]:
combined

,actual,predicted
55,1,1
56,1,1
57,1,1
58,1,0
59,1,0
...,...,...
1447,0,0
1448,1,0
1449,0,0
1450,0,0


In [146]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index = True)

In [147]:
combined

,actual,predicted,date,team,opponent,result
55,1,1,2024-01-20,Arsenal,Crystal Palace,W
56,1,1,2024-01-30,Arsenal,Nott'ham Forest,W
57,1,1,2024-02-04,Arsenal,Liverpool,W
58,1,0,2024-02-11,Arsenal,West Ham,W
59,1,0,2024-02-17,Arsenal,Burnley,W
...,...,...,...,...,...,...
1447,0,0,2024-04-24,Wolverhampton Wanderers,Bournemouth,L
1448,1,0,2024-04-27,Wolverhampton Wanderers,Luton Town,W
1449,0,0,2024-05-04,Wolverhampton Wanderers,Manchester City,L
1450,0,0,2024-05-11,Wolverhampton Wanderers,Crystal Palace,L


In [159]:
unique_teams = combined['team'].unique()
print(unique_teams)

other = combined["opponent"].unique()
print(other)

['Arsenal' 'Aston Villa' 'Bournemouth' 'Brentford'
 'Brighton and Hove Albion' 'Burnley' 'Chelsea' 'Crystal Palace' 'Everton'
 'Fulham' 'Liverpool' 'Luton Town' 'Manchester City' 'Manchester United'
 'Newcastle United' 'Nottingham Forest' 'Sheffield United'
 'Tottenham Hotspur' 'West Ham United' 'Wolverhampton Wanderers']
['Crystal Palace' "Nott'ham Forest" 'Liverpool' 'West Ham' 'Burnley'
 'Newcastle Utd' 'Sheffield Utd' 'Brentford' 'Manchester City'
 'Luton Town' 'Brighton' 'Aston Villa' 'Wolves' 'Chelsea' 'Tottenham'
 'Bournemouth' 'Manchester Utd' 'Everton' 'Fulham' 'Arsenal']


In [151]:
#normalizng names across entire dataframe bet team and opponent cols

class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {
    "Brighton and Hove Albion": "Brighton",
    "Manchester United": "Manchester Utd",
    "Newcastle United": "Newcastle Utd",
    "Tottenham Hotspur":"Tottenham",
    "West Ham United": "West Ham",
    "Wolverhampton Wanderers": "Wolves",
    "Newcastle United": "Newcastle Utd",
    "Nottingham Forest": "Nott'ham Forest"
}

mapping = MissingDict(**map_values)

In [153]:
mapping["West Ham United"]

'West Ham United'

In [161]:
combined["new_team"] = combined["team"].map(mapping)

In [162]:
merged = combined.merge(combined, left_on = ["date", "new_team"], right_on=["date", "opponent"])
#merge results on either side of the draw

In [163]:
merged

,actual_x,predicted_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,predicted_y,team_y,opponent_y,result_y,new_team_y
0,1,1,2024-01-20,Arsenal,Crystal Palace,W,Arsenal,0,0,Crystal Palace,Arsenal,L,Crystal Palace
1,1,1,2024-01-30,Arsenal,Nott'ham Forest,W,Arsenal,0,0,Nottingham Forest,Arsenal,L,Nottingham Forest
2,1,1,2024-02-04,Arsenal,Liverpool,W,Arsenal,0,0,Liverpool,Arsenal,L,Liverpool
3,1,0,2024-02-11,Arsenal,West Ham,W,Arsenal,0,0,West Ham United,Arsenal,L,West Ham United
4,1,0,2024-02-17,Arsenal,Burnley,W,Arsenal,0,0,Burnley,Arsenal,L,Burnley
...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,0,0,2024-04-24,Wolverhampton Wanderers,Bournemouth,L,Wolves,1,0,Bournemouth,Wolves,W,Bournemouth
307,1,0,2024-04-27,Wolverhampton Wanderers,Luton Town,W,Wolves,0,0,Luton Town,Wolves,L,Luton Town
308,0,0,2024-05-04,Wolverhampton Wanderers,Manchester City,L,Wolves,1,1,Manchester City,Wolves,W,Manchester City
309,0,0,2024-05-11,Wolverhampton Wanderers,Crystal Palace,L,Wolves,1,1,Crystal Palace,Wolves,W,Crystal Palace


In [167]:
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"]==0)]["actual_x"].value_counts()

actual_x
1    59
0    39
Name: count, dtype: int64

In [168]:
59/(59+39)

0.6020408163265306